In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from scipy.sparse import hstack  
from ast import literal_eval

In [2]:
df = pd.read_csv("C:\\Users\\20100\\Desktop\\graduation project\\ML Model\\furniture_store - Copy_db.csv",
                 usecols=["product_id","product_category","product_name","product_description"])
# df = pd.read_csv("C:\\Users\\20100\\Desktop\\graduation project\\ML Model\\roveconcepts2.csv",
#                  usecols=["product_category","product_name","product_description"])

In [3]:
df.head()

,product_id,product_category,product_name,product_description
0,30180504,Bar furniture,NORBERG,"Wall-mounted drop-leaf table, ..."
1,10122647,Bar furniture,INGOLF,"Bar stool with backrest, 63 cm"
2,70404875,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
3,50406465,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
4,90404879,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."


In [4]:
if '30180504' in df['product_id'].astype(str).values:
    print("Value found in DataFrame.")
else:
    print("Value not found in DataFrame.")

Value found in DataFrame.


In [5]:
df = df.set_index('product_id')

In [6]:
df["product_name"].value_counts()

PAX                  101
BESTÅ                 96
EKET                  54
PLATSA                53
GRÖNLID               52
                    ... 
TERJE                  1
EKENÄS                 1
SVENARNE               1
FANBYN                 1
PAX / MEHAMN/AULI      1
Name: product_name, Length: 288, dtype: int64

### some of products name is redundant in many categories ,so it is not good to recommend by 'products_name'             ,So it is better to use product_id 

In [74]:
# Define the product description and category/name vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = CountVectorizer(stop_words='english')

product_descriptions = df['product_description']
product_categories = df['product_category']
product_names = df['product_name']

# Create the product description and category/name matrices
tfidf_matrix = tfidf_vectorizer.fit_transform(product_descriptions)
count_matrix = count_vectorizer.fit_transform(product_categories + ' ' + product_names)

# Combine the matrices
combined_similarity_matrix = cosine_similarity(hstack([tfidf_matrix, count_matrix]))

In [75]:
######modify######
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = CountVectorizer(stop_words='english')

product_descriptions = df['product_description']
product_categories = df['product_category']
product_names = df['product_name']

tfidf_matrix = tfidf_vectorizer.fit_transform(product_descriptions)

# Apply a weight factor to the count matrix for 'product_category'
category_matrix_weight = 2  # Set the weight factor for 'product_category'
count_matrix_category = count_vectorizer.fit_transform(category_matrix_weight * product_categories)
count_matrix_name = count_vectorizer.fit_transform(product_names)
count_matrix = hstack([count_matrix_category, count_matrix_name])

# Combine the matrices
combined_similarity_matrix = cosine_similarity(hstack([tfidf_matrix, count_matrix]))

In [76]:
def get_related_products(product_name, num_products=25):
    # Find the index of the input product
    if product_name not in df['product_name'].values:
        print(f"{product_name} not found in product list.")
        return None
    idx = df[df['product_name']==product_name].index[0]

    # Get the cosine similarity scores of all products to the input product
    sim_scores = list(enumerate(combined_similarity_matrix[idx]))

    # Sort the products by their similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top num_products similar products
    sim_indices = [i[0] for i in sim_scores[1:num_products+1]]

    # Return the top num_products similar products
    return df.iloc[sim_indices][['product_category', 'product_name', 'product_description']]

related_products = get_related_products('HEMNES')
related_products

,product_category,product_name,product_description
35,Beds,HEMNES,"Day-bed frame with 3 drawers, ..."
41,Beds,HEMNES,"Day-bed w 3 drawers/2 mattresses, ..."
48,Beds,BRIMNES,"Day-bed frame with 2 drawers, ..."
62,Beds,BRIMNES,"Day-bed w 2 drawers/2 mattresses, ..."
23,Beds,PLATSA,"Bed frame with 4 drawers, 140..."
52,Beds,PLATSA,"Bed frame with 2 drawers, 142..."
59,Beds,PLATSA,"Bed frame with 4 drawers, 142..."
1239,Sofas & armchairs,HEMNES,"Day-bed w 3 drawers/2 mattresses, ..."
53,Beds,PLATSA,"Bed frame with 4 doors+6 drawers, ..."
56,Beds,PLATSA,"Bed frame with 2 door+3 drawers, ..."


In [7]:
from scipy.sparse import hstack

# Define the product description and category/name vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = CountVectorizer(stop_words='english')

product_descriptions = df['product_description']
product_categories = df['product_category']
product_names = df['product_name']

# Create the product description and category/name matrices
tfidf_matrix = tfidf_vectorizer.fit_transform(product_descriptions)

# Apply a weight factor to the count matrix for 'product_category'
category_matrix_weight = 2  # Set the weight factor for 'product_category'
count_matrix_category = count_vectorizer.fit_transform(category_matrix_weight * product_categories)
count_matrix_name = count_vectorizer.fit_transform(product_names)
count_matrix = hstack([count_matrix_category, count_matrix_name])

# Combine the matrices
combined_similarity_matrix = cosine_similarity(hstack([tfidf_matrix, count_matrix]))

def get_related_products(product_id, num_products=25):
    # Find the index of the input product
    if product_id not in df.index:
        print(f"{product_id} not found in product list.")
        return None
    idx = df.index.get_loc(product_id)

    # Get the cosine similarity scores of all products to the input product
    sim_scores = list(enumerate(combined_similarity_matrix[idx]))

    # Sort the products by their similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top num_products similar products
    sim_indices = [i[0] for i in sim_scores[1:num_products+1]]

    # Return the top num_products similar products
    return df.iloc[sim_indices][['product_category', 'product_name', 'product_description']]

related_products = get_related_products(10122647)
related_products

,product_category,product_name,product_description
product_id,,,
121766,Bar furniture,INGOLF,"Bar stool with backrest, 74 cm"
30352246,Bar furniture,RÅSKOG,"Bar stool, 63 cm"
30308395,Bar furniture,RÅSKOG,"Bar stool, 63 cm"
70404875,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
50406465,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
397736,Bar furniture,NORRARYD,"Bar stool with backrest, 74 cm"
90319918,Bar furniture,HENRIKSDAL,"Bar stool with backrest, 74 cm"
39184324,Bar furniture,HENRIKSDAL,"Bar stool with backrest, 74 cm"
20429010,Bar furniture,NORRÅKER,"Bar stool with backrest, 74 cm"


1-First, it checks if the input product_id exists in the index of the DataFrame df.
  If the product_id is not found, the function prints a message and returns None.
2-If the product_id is found, the function gets the index idx of the input product in the DataFrame df.
3-The function then gets the cosine similarity scores between the input product and all other products in the DataFrame df.
  It does this by getting the idx-th row of the combined_similarity_matrix.
3-The similarity scores are then sorted in descending order, so that the most similar products are first.
4-The function then selects the top num_products similar products (excluding the input product itself)
  by getting the indices of the products with the highest similarity scores. 
    These indices are stored in the sim_indices variable.
5-Finally, the function returns a DataFrame containing the product category, name, and description
  of the top num_products similar products, as indexed by sim_indices.

In [12]:
from scipy.sparse import hstack

# Set 'product_id' as the index of the DataFrame
# df = df.set_index('product_id')

# Define the product description and category/name vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = CountVectorizer(stop_words='english')

product_descriptions = df['product_description']
product_categories = df['product_category']
product_names = df['product_name']

# Create the product description and category/name matrices
tfidf_matrix = tfidf_vectorizer.fit_transform(product_descriptions)

# Apply a weight factor to the count matrix for 'product_category'
category_matrix_weight = 2  # Set the weight factor for 'product_category'
count_matrix_category = count_vectorizer.fit_transform(category_matrix_weight * product_categories)
count_matrix_name = count_vectorizer.fit_transform(product_names)
count_matrix = hstack([count_matrix_category, count_matrix_name])

# Combine the matrices
combined_similarity_matrix = cosine_similarity(hstack([tfidf_matrix, count_matrix]))


# Reset index of our main DataFrame and construct reverse mapping as before
df = df.reset_index()
indices = pd.Series(df.index, index=df['product_id'])

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=combined_similarity_matrix):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['product_id'].iloc[product_indices]

In [125]:
get_recommendations(30275861)

1075     9017483
1073    70301537
177     10275862
275     89030584
307     19030592
340     39017486
364     19278328
393     69017475
109     80275887
140     70301542
Name: product_id, dtype: int64